In [ ]:
!apt-get update
!apt-get install -y libopencv-dev python3-opencv


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 2s (160 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package list

In [ ]:
import cv2
print(cv2.__version__)


4.10.0


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving redCostume.jpeg to redCostume (1).jpeg


In [ ]:
%%writefile image_conversion_cpu.cpp


#include<stdio.h>
#include<opencv2/opencv.hpp>  //openCV header
#include<chrono>              // for timing

using namespace cv;         // openCV namespace
using namespace std::chrono;    // for timing

//function to convert RGB to Grayscale (cpu version)

void rgb_to_greyscale_cpu(unsigned char *input,unsigned char *output,int width,int height){
  for(int y = 0; y < height ; y++){
    for(int x = 0; x < width ; x++){
      int idx = (y * width + x)*3;  // 3 channels (B,G,R)

      unsigned char r = input[idx + 2];
      unsigned char g = input[idx + 1];
      unsigned char b = input[idx];

      // Apply the formula: L = 0.21*R + 0.72*G + 0.07*B
      unsigned char L = (unsigned char)(0.21f * r + 0.72f * g + 0.07f * b );
      output[y * width + x] = L;
    }
  }
}

int main(){
  // Load the image using openCV

  Mat img = imread("redConstume.jpeg",IMREAD_COLOR);
  if(img.empty()){
    printf("Error: Could not open or find the image.\n");
   // return -1;
  }

  int width = img.cols;
  int height = img.rows;
  int img_size = width * height * 3;    // for RGB (or BGR) image
  int grey_img_size = width * height;   // for grayscale image

  //Allocate memory for grayscale output
  unsigned char *h_output = (unsigned char*)malloc(grey_img_size);    // Host output image(Grayscale)

  //OpenCV stores datea in BGR format by default
  unsigned char *h_input = img.data;    // openCV Math image data pointer (BGR)

  //Start timing
  auto start = high_resolution_clock::now();

  //Perform RGB to grayscale conversion on CPU
  reg_to_greyscale_cpu(h_input,h_output,width,height);

  //stop timing

  auto stop = high_resolution_clock::now();
  auto duration = duration_cast<milliseconds>(stop-start);

  printf("Time for grayscale conversion (CPU): %lld ns\n",duration.count());

  //save the grayscale image using openCV
  Mat grey_img(height,width,CV_8UC1,h_output);
  imwrite("greyscale-tiger.jpeg",grey_img);

  //free host memory
  free(h_output);

  return 0;
}

Overwriting image_conversion_cpu.cpp


In [ ]:
!g++ image_conversion_cpu.cpp -o image_conversion_cpu 'pkg-config --cflags --libs opencv4'


image_conversion_cpu.cpp:3:9: fatal error: opencv2/opencv.hpp: No such file or directory
    3 | #include<opencv2/opencv.hpp>  //openCV header
      |         ^~~~~~~~~~~~~~~~~~~~
compilation terminated.


In [ ]:
#download converted image
from google.colab import files
files.download("greyscale-tiger.jpeg")

FileNotFoundError: Cannot find file: greyscale-tiger.jpeg

In [ ]:
%%writefile image_conversion_gpu.cu

#include<stdio.h>
#include<cuda_runtime.h>              // for timing
#include<opencv2/opencv.hpp>  //openCV header

using namespace cv;

//CUDA kernel to convert RGB to Grayscale
__global__ void rgb_to_greyscale(unsigned char *input,unsigned char *output, int width,int height){
  int x = blockIdx.x * blockDim.x + threadIdx.x;
  int y = blockIdx.y * blockDim.y + threadIdx.y;
  int idx = (y * width + x) * 3;  // 3 channels(R,G,B)

  if(x < width && y < height){
    unsigned char r = input[idx];       // red channel
    unsigned char g = input[idx + 1];   // Green channel
    unsigned char b = input[idx + 2];   //  Blue channel

    // Apply the formula : L = 0.21*R + 0.72*G + 0.07*B

    unsigned char L = (unsigned char)(0.21f * r + 0.72f * g + 0.07f * b);
    output[y * width + x] = L;

  }
}

int main(){
  //load the image using openCV
  Mat img = imread("colorful-tiger.jpeg",IMREAD_COLOR);
  if(img.empty()){
    printf("Error: could not open or find the image.\n");
    return -1;
  }

  int width = img.cols;
  int height = img.rows;
  int img_size = width * height * 3;    // For RGB (or BGR ) image
  int grey_img_size = width * height;   // For grayscale image

  // Allocate memory on the host for grayscale output
  unsigned char *h_output = (unsigned char*)malloc(grey_img_size);    // Host output image (Grayscale)

  //openCV stores data in BGR format by default
  unsigned char *h_input = img.data;    // openCV Mat image data pointer

  unsigned char *d_input, *d_output;
  cudaMalloc((void**)&d_input, img_size);
  cudaMalloc((void**)&d_output, grey_img_size);

  // Create CUDA events for timing
  cudaEvent_t start,stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  cudaEventRecord(start);

  cudaMemcpy(d_input,h_input,img_size,cudaMemcpyHostToDevice);

  dim3 threadsPerBlock(16,16);
  dim3 blocksPerGrid((width + 15) / 16 , (height + 15) / 16);

  rgb_to_greyscale<<<blocksPerGrid,threadsPerBlock>>>(d_input,d_output,width,height);

  cudaMemcpy(h_output, d_output, grey_img_size , cudaMemcpyDeviceToHost);

  cudaEventSynchronize(stop);
  cudaEventRecord(stop);

  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds,start,stop);
  printf("Time for grayscale conversion (CPU): %f ms\n", milliseconds);

  Mat grey_img(height,width,CV_8UC1,h_output);
  imwrite("greyscale-tiger.jped",grey_img);

  cudaFree(d_input);
  cudaFree(d_output);

  free(h_output);

  return 0;
}

Overwriting image_conversion_gpu.cu


In [ ]:
!nvcc -o image_conversion_gpu image_conversion_gpu.cu `pkg-config --cflags --libs opencv4`

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is

In [ ]:
!./image_conversion_gpu

Error: could not open or find the image.
